# Selección, entrenamiento y evaluación de un modelo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# %cd '/content/drive/MyDrive/Inteligencia Artificial/IA - Clases de Práctica/PRACTICA_2023_2C/codigos_de_clase'

In [4]:
np.random.seed(42)

# Cargamos los datos
df_housing = pd.read_csv("./data/housing.csv")

# Creamos nuestro atributo categórico para los ingresos
df_housing["income_cat"] = pd.cut(df_housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])

# Dividimos los datos en conjunto de entrenamiento y prueba
strat_train_set, strat_test_set = train_test_split(df_housing, test_size=0.2, stratify=df_housing["income_cat"], random_state=42)

# Eliminamos la categoria income_cat de ambos conjuntos porque no la usamos
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Separamos predictores y etiquetas
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [5]:
num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
])

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))


log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler())


preprocessing = ColumnTransformer(
    [
        ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                               "households", "median_income"]),
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
    ],
    remainder=num_pipeline

    )

## Selección y Entrenamiento

¡Finalmente! Hemos resuelto el problema, obtuvimos los datos, los exploramos, los dividimos en un conjunto de entrenamiento y un conjunto de prueba, y creamos un _pipeline_ de preprocesamiento para limpiar y preparar automáticamente los datos para los algoritmos de aprendizaje maquinal. Ahora estamos listos para seleccionar y entrenar un modelo.

In [6]:
from sklearn.linear_model import LinearRegression

lin_reg = make_pipeline(preprocessing, LinearRegression())
lin_reg.fit(housing, housing_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('impute',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardize',
                                                              StandardScaler())]),
                                   transformers=[('log',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out='one-to-one',
                                                                                       func=<ufunc 'log'>)),
                                                                  ('standa...
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002241EC78390>)])),
                ('linearregression', LinearRegression())])

In [7]:
housing_predictions = lin_reg.predict(housing)
housing_predictions[:5].round()

array([251401., 355841., 127167., 107640., 295921.])

In [8]:
housing_labels.iloc[:5].values

array([458300., 483800., 101700.,  96100., 361800.])

Vamos a usar la raiz del error cuadrático medio (RMSE) como medida de desempeño

In [9]:
from sklearn.metrics import mean_squared_error

lin_rmse = mean_squared_error(housing_labels, housing_predictions, squared=False)
lin_rmse

71384.69747107038

El valor de `median_house_value` de los distritos oscila entre $120,000 y $265,000, por lo que un error de predicción de $71,384 no es muy bueno. Este es un ejemplo de un modelo mal ajustado (`underfitting`) a los datos de entrenamiento. Cuando esto ocurre, puede significar que las características no proporcionan suficiente información para realizar buenas predicciones o que el modelo no es lo suficientemente bueno.

Formas de corregir el desajuste son seleccionar un modelo más potente, proporcionar al algoritmo mejores características o reducir las restricciones en el modelo.

In [10]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_reg.fit(housing, housing_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('impute',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardize',
                                                              StandardScaler())]),
                                   transformers=[('log',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out='one-to-one',
                                                                                       func=<ufunc 'log'>)),
                                                                  ('standa...
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002241EC78390>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [11]:
housing_predictions = tree_reg.predict(housing)
tree_rmse = mean_squared_error(housing_labels, housing_predictions,squared=False)
tree_rmse

0.0

¿Ningún error en absoluto? Es probable que el modelo se haya sobreajustado (`overfitting`) a los datos. La situación más probable es que el modelo haya memorizado los datos de entrenamiento en lugar de aprender patrones generales que puedan aplicarse a nuevos datos. El sobreajuste ocurre cuando un modelo se ajusta demasiado a los detalles específicos de los datos de entrenamiento y, como resultado, tiene un rendimiento deficiente en datos nuevos y no vistos.

¿Cómo podemos comprobarlo? ¿Usamos los datos de prueba? Como dijimos anteriormente, no miramos el conjunto de prueba ni lo usamos, hasta que un modelo entrenado esté listo para usar, por lo que necesitas utilizar parte del conjunto de entrenamiento para el entrenamiento y otra parte para la validación del modelo.

## Validación del Modelo

Usamos validación cruzada

In [12]:
from sklearn.model_selection import cross_val_score

tree_rmses = -cross_val_score(tree_reg, housing, housing_labels, scoring="neg_root_mean_squared_error", cv=10)

In [13]:
pd.Series(tree_rmses).describe()

count       10.000000
mean     68395.620297
std       1519.009761
min      66645.921836
25%      67207.104286
50%      68209.372643
75%      69636.965788
max      71048.558253
dtype: float64

In [14]:
lin_rmses = -cross_val_score(lin_reg, housing, housing_labels,
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(lin_rmses).describe()

count       10.000000
mean     71483.718581
std       1377.115270
min      69197.929928
25%      70795.360185
50%      71707.200873
75%      72337.050164
max      73732.501674
dtype: float64

In [15]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = make_pipeline(preprocessing, RandomForestRegressor(random_state=42))
forest_rmses = -cross_val_score(forest_reg, housing, housing_labels, scoring="neg_root_mean_squared_error", cv=10)
pd.Series(forest_rmses).describe()

count       10.000000
mean     48856.086407
std        720.912468
min      47418.298531
25%      48496.412362
50%      48801.997453
75%      49315.093913
max      50001.052410
dtype: float64

## Ajuste de hiperparámetros del modelo

Usaremos GridSearch para ajustar los parámetros de nuestros modelos (Lo veremos en las siguientes clases)

## Evaluación con los datos de prueba

In [18]:
final_model = forest_reg
forest_reg.fit(housing, housing_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('impute',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardize',
                                                              StandardScaler())]),
                                   transformers=[('log',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out='one-to-one',
                                                                                       func=<ufunc 'log'>)),
                                                                  ('standa...
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002241EC78390>)])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [19]:
final_model = forest_reg

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

final_predictions = final_model.predict(X_test)

final_rmse = mean_squared_error(y_test, final_predictions, squared=False)
print(final_rmse)

49538.17427244866
